In [2]:
import json,csv,pickle

In [55]:
with open('images.tsv', mode ='r',encoding='utf8')as file:
    lines = file.readline()
    assert lines.strip().startswith('UUID\tparameters'), "Data format error!"
    lines = file.readlines()

In [56]:
lines

['0216077-d5d4-4e63-beb4-3a392c93f887\t{"Positive": "high quality,\\thighres,1girl,cloud, scenery, sunset", "Negative": "", "Ratio": "2", "Sampler": "DPM2KA", "Seed": "-1"}\n',
 '246ac8ac-9155-4a84-a17f-797f91dde1ad\t{"Positive": "high quality,highres,\t1girl,cloud, scenery, sunset", "Negative": "", "Ratio": "2", "Sampler": "DPM2KA", "Seed": "-1"}\n',
 '182fe3d1-5709-46a0-8ad2-c16c2bc316df\t{"Positive": "high quality,highres,1girl,cloud, scenery, sunset", "Negative": "", "Ratio": "2", "Sampler": "DPM2KA", "Seed": "-1"}']

In [59]:
imageDic = {}
for line in lines:
    cache = line.strip().split('\t')
    imageDic[cache[0]] = json.loads(' '.join(cache[1:]))

In [60]:
imageDic

{'0216077-d5d4-4e63-beb4-3a392c93f887': {'Positive': 'high quality,\thighres,1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'},
 '246ac8ac-9155-4a84-a17f-797f91dde1ad': {'Positive': 'high quality,highres, 1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'},
 '182fe3d1-5709-46a0-8ad2-c16c2bc316df': {'Positive': 'high quality,highres,1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'}}

In [3]:
imageDic = {}
with open('images.tsv', mode ='r',encoding='utf8')as file:
    line = file.readline()
    assert line.strip().startswith('UUID\tparameters'), "Data format error!"
    line = file.readline()
    while line:
        cache = line.strip().split('\t')
        imageDic[cache[0]] = json.loads(' '.join(cache[1:]))
        line = file.readline()

In [4]:
imageDic

{'0216077-d5d4-4e63-beb4-3a392c93f887': {'Positive': 'high quality,\thighres,1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'},
 '246ac8ac-9155-4a84-a17f-797f91dde1ad': {'Positive': 'high quality,highres, 1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'},
 '182fe3d1-5709-46a0-8ad2-c16c2bc316df': {'Positive': 'high quality,highres,1girl,cloud, scenery, sunset',
  'Negative': '',
  'Ratio': '2',
  'Sampler': 'DPM2KA',
  'Seed': '-1'}}

In [6]:
line ,cache = 0,0

In [8]:
cache

0

In [ ]:
def storeImg(img,params):
    pass